In [27]:
import requests
from bs4 import BeautifulSoup
from pymongo import MongoClient

In [28]:
# connect pymongo
client = MongoClient("mongodb://localhost:27017")
# client.list_database_names()
# connect Database
db = client["May"]
# connect collection
col = db["0506"]

In [44]:
# 設定要爬取的網址
url = 'https://www.skyscanner.com.tw/transport/flights/tpet/kix/230511/230518/?adults=1&adultsv2=1&cabinclass=economy&children=0&childrenv2=&destinationentityid=27542908&inboundaltsenabled=false&infants=0&originentityid=27547236&outboundaltsenabled=false&preferdirects=false&ref=home&rtn=1'

# 發送 GET 請求並取得 HTML 網頁原始碼
response = requests.get(url)
html = response.text

# 解析 HTML 網頁原始碼
soup = BeautifulSoup(response.text, 'html.parser')

# print(soup.prettify())

In [51]:
import requests
import json
from pymongo import MongoClient

# 設定要爬取的起點和終點城市代碼、出發日和返程日
origin = 'TPE'
destination = 'KIX'
outbound_date = '2022-05-20'
inbound_date = '2022-05-25'

# 設定 API 的 URL 和參數
url = 'https://skyscanner-skyscanner-flight-search-v1.p.rapidapi.com/apiservices/browsedates/v1.0/TW/TWD/zh-TW/{}/{}/{}/{}/'.format(origin, destination, outbound_date, inbound_date)
headers = {
    'X-RapidAPI-Host': 'skyscanner-skyscanner-flight-search-v1.p.rapidapi.com',
    'X-RapidAPI-Key': 'prtl6749387986743898559646983194'
}

# 發送 GET 請求並取得 JSON 資料
response = requests.get(url, headers=headers)
data = response.json()

print(response.text)

{"message":"You are not subscribed to this API."}


In [68]:
import requests
import json
from pymongo import MongoClient

# 設定要爬取的起點和終點城市代碼、出發日和返程日
origin = 'TPE'
destination = 'KIX'
outbound_date = '2022-05-20'
inbound_date = '2022-05-25'

# 設定 API 的 URL 和參數
url = 'https://partners.api.skyscanner.net/apiservices/v3/flights/live/search/create'
headers = {
    'x-api-key': 'prtl6749387986743898559646983194',
    'Content-Type': 'application/json'
}
params = {
    'body': {
        'query': {
            'originPlace': origin,
            'destinationPlace': destination,
            'outboundDate': outbound_date,
            'inboundDate': inbound_date,
            'cabinClass': 'economy',
            'adults': 1,
            'children': 0,
            'infants': 0,
            'locale': 'en-US',
            'currency': 'USD',
            'flightType': 'roundtrip'
        }
    }
}

# 發送 POST 請求並取得 JSON 資料
response = requests.post(url, headers=headers, data=json.dumps(params))
data = response.json()

print(response.text)

{"code":3,"message":"The currency is missing\nThe locale is missing\nThe market is missing\nThe query leg list must contain at least 1 leg\nThe number of adults must be between 1 and 8\nThe cabin class is invalid","details":[]}


In [66]:
# 將 JSON 資料存入 MongoDB
for quote in data['Quotes']:
    doc = {
        'carrier_name': next((carrier['Name'] for carrier in data['Carriers'] if carrier['CarrierId'] == quote['OutboundLeg']['CarrierIds'][0]), None),
        'outbound_date': quote['OutboundLeg']['DepartureDate'],
        'inbound_date': quote['InboundLeg']['DepartureDate'],
        'price': quote['MinPrice']
    }
    col.insert_one(doc)


KeyError: 'Quotes'

In [63]:
# 了解發送的請求的詳細資訊，查找錯誤或者進行調試。
print(response.request.__dict__)

{'method': 'POST', 'url': 'https://partners.api.skyscanner.net/apiservices/v3/flights/live/search/create', 'headers': {'User-Agent': 'python-requests/2.28.1', 'Accept-Encoding': 'gzip, deflate', 'Accept': '*/*', 'Connection': 'keep-alive', 'x-api-key': 'prtl6749387986743898559646983194', 'Content-Type': 'application/json', 'Content-Length': '261'}, '_cookies': <RequestsCookieJar[]>, 'body': '{"body": {"query": {"originPlace": "TPE", "destinationPlace": "KIX", "outboundDate": "2022-05-20", "inboundDate": "2022-05-25", "cabinClass": "economy", "adults": 1, "children": 0, "infants": 0, "locale": "en-US", "currency": "USD", "flightType": "roundtrip"}}}', 'hooks': {'response': []}, '_body_position': None}


In [62]:
print(data)

{'code': 8, 'message': '', 'details': []}


In [56]:
# 連接 MongoDB
client = MongoClient('mongodb://localhost:27017/')
db = client['May']
collection = db['0506']

In [61]:
# 將 JSON 資料存入 MongoDB
for quote in data['Quotes']:
    doc = {
        'carrier_name': next((carrier['Name'] for carrier in data['Carriers'] if carrier['CarrierId'] == quote['OutboundLeg']['CarrierIds'][0]), None),
        'outbound_date': quote['OutboundLeg']['DepartureDate'],
        'inbound_date': quote['InboundLeg']['DepartureDate'],
        'price': quote['MinPrice']
    }
    collection.insert_one(doc)

print('Done!')

KeyError: 'Quotes'

In [71]:
import requests
from bs4 import BeautifulSoup
from pymongo import MongoClient

# 設定要爬取的起點和終點城市代碼、出發日和返程日
outbound_date = '04/06/2023'
inbound_date = '10/06/2023'

# 設定要爬取的網址和參數
url = 'https://flight.eztravel.com.tw/tickets-roundtrip-tpe-osa/'
params = {
    'outbounddate': outbound_date,
    'inbounddate': inbound_date,
    'adults': '1',
    'children': '0',
    'infants': '0',
    'direct': 'false',
    'cabintype': 'any',
    'searchbox': 't'
}

In [72]:
# 發送 GET 請求並取得 HTML 頁面
response = requests.get(url, params=params)
html = response.text

print(response.text)

<!DOCTYPE html><html lang="zh-Hant-TW" class="wide"><head><meta charset="utf-8"><meta http-equiv="X-UA-Compatible" content="IE=edge,chrome=1"><meta name="viewport" content="width=device-width"><title>台北TPE - 大阪OSA 來回機票搜尋 | ezTravel易遊網</title><link rel="icon" type="image/x-icon" href="https://static.cdn-eztravel.com/assets/images/common/favicon.ico"><script>(function(w, d, s, l, i) {
            w[l] = w[l] || [];
            w[l].push({
                'gtm.start': new Date().getTime(),
                event: 'gtm.js'
            });
            var f = d.getElementsByTagName(s)[0],
                j = d.createElement(s),
                dl = l != 'dataLayer' ? '&l=' + l : '';
            j.async = true;
            j.src = 'https://www.googletagmanager.com/gtm.js?id=' + i + dl;
            f.parentNode.insertBefore(j, f);
        })(window, document, 'script', 'dataLayer', 'GTM-P3C3D8V');</script><noscript><iframe src="https://www.googletagmanager.com/ns.html?id=GTM-P3C3D8V" height="0

In [74]:
# 使用 BeautifulSoup 解析 HTML 頁面並提取機票資訊
soup = BeautifulSoup(html, 'html.parser')
flight_list = soup.select('#section-1 .flight-card')

In [75]:
# 存入 MongoDB 中
client = MongoClient('mongodb://localhost:27017/')
db = client['May']
col = db['0506']

In [79]:
for flight in flight_list:
    data = {
        'airline': flight.select_one('.airline').text.strip(),
        'departure_time': flight.select_one('.dep-time').text.strip(),
        'arrival_time': flight.select_one('.arr-time').text.strip(),
        'price': flight.select_one('.price').text.strip()
    }
    col.insert_many(data)

print(data)

{'code': 3, 'message': 'The currency is missing\nThe locale is missing\nThe market is missing\nThe query leg list must contain at least 1 leg\nThe number of adults must be between 1 and 8\nThe cabin class is invalid', 'details': []}


In [81]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

url = 'https://flight.eztravel.com.tw/tickets-roundtrip-tpe-osa/?outbounddate=04%2F06%2F2023&inbounddate=10%2F06%2F2023&dport=&aport=&adults=1&children=0&infants=0&direct=false&cabintype=any&airline=&searchbox=t'

response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')

flight_list = soup.select('.flight-item')
data = []
for flight in flight_list:
    row = {
        'airline': flight.select_one('.airline').text.strip(),
        'departure_time': flight.select_one('.dep-time').text.strip(),
        'arrival_time': flight.select_one('.arr-time').text.strip(),
        'price': flight.select_one('.price').text.strip()
    }
    data.append(row)

df = pd.DataFrame(data)
print(df)

Empty DataFrame
Columns: []
Index: []


In [87]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

# 設定要爬取的網址
url = 'https://flight.eztravel.com.tw/tickets-roundtrip-tpe-osa/?outbounddate=04%2F06%2F2023&inbounddate=10%2F06%2F2023&dport=&aport=&adults=1&children=0&infants=0&direct=false&cabintype=any&airline=&searchbox=t'

# 連接網頁，取得HTML原始碼
response = requests.get(url)
html = response.text

# 使用BeautifulSoup解析HTML原始碼，取得所需的資料
soup = BeautifulSoup(html, 'html.parser')
flight_list = soup.find_all("div", class_="flight-list-item clearfix")

# 將資料儲存到DataFrame
data = []
for flight in flight_list:
    row = {
        'airline': flight.select_one('.airline').text.strip(),
        'departure_time': flight.select_one('.dep-time').text.strip(),
        'arrival_time': flight.select_one('.arr-time').text.strip(),
        'price': flight.select_one('.price').text.strip()
    }
    data.append(row)
df = pd.DataFrame(data)

print(df.head())


Empty DataFrame
Columns: []
Index: []


In [91]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

url = 'https://flight.eztravel.com.tw/tickets-roundtrip-tpe-osa/?outbounddate=04%2F06%2F2023&inbounddate=10%2F06%2F2023&dport=&aport=&adults=1&children=0&infants=0&direct=false&cabintype=any&airline=&searchbox=t'

# 發送 GET 請求
response = requests.get(url)

# 解析 HTML
soup = BeautifulSoup(response.text, 'html.parser')

# 找到所有的航班列表項目
flight_list = soup.select('.flight-list-item')

# 提取機票資訊並存成 DataFrame
data = []

for flight in flight_list:
    airline = flight.select_one('.airline').text.strip()
    departure_time = flight.select_one('.dep-time').text.strip()
    arrival_time = flight.select_one('.arr-time').text.strip()
    price = flight.select_one('.price').text.strip()

    data.append({'airline': airline,
                 'departure_time': departure_time,
                 'arrival_time': arrival_time,
                 'price': price})

df = pd.DataFrame(data)
print(df)

Empty DataFrame
Columns: []
Index: []


In [92]:
import json
import bs4
import os
import urllib.request as req
import requests
import pandas as pd 


In [96]:
!pip3 install selenium

     ---------------------------------------- 6.5/6.5 MB 7.6 MB/s eta 0:00:00
  Using cached trio-0.22.0-py3-none-any.whl (384 kB)
  Using cached trio_websocket-0.10.2-py3-none-any.whl (17 kB)
  Using cached exceptiongroup-1.1.1-py3-none-any.whl (14 kB)
  Using cached outcome-1.2.0-py2.py3-none-any.whl (9.7 kB)
     ------------------------------------- 179.1/179.1 kB 11.3 MB/s eta 0:00:00
  Using cached async_generator-1.10-py3-none-any.whl (18 kB)
  Using cached wsproto-1.2.0-py3-none-any.whl (24 kB)
     -------------------------------------- 118.7/118.7 kB 6.8 MB/s eta 0:00:00



[notice] A new release of pip available: 22.3.1 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [99]:
from selenium import webdriver
import time

options = webdriver.ChromeOptions()
options.add_argument('headless') # 可以設置為不顯示瀏覽器
driver = webdriver.Chrome(options=options)

def scroll(scrolltimes):
  for i in range(scrolltimes):
    # 每一次頁面滾動都是滑到網站最下方
    js = 'window.scrollTo(0, document.body.scrollHeight);'
    driver.execute_script(js)
    time.sleep(2)

In [118]:
import json
import bs4
import os
import urllib.request as req

import requests

url = 'https://flight.eztravel.com.tw/apiv2/flight/list?sig=3841c4acaff594b8'
response = requests.post(url)

print(soup.prettify())

{"timestamp":"2023-05-07T12:39:01.901+0000","status":400,"error":"Bad Request","message":"Required request body is missing: public tw.com.eztravel.flight.flightlist.CommonResponse
<tw.com.eztravel.flight.flightlist.getflightlistresponse>
 tw.com.eztravel.controller.FlightController.getFlightList(tw.com.eztravel.flight.flightlist.CommonRequest
 <tw.com.eztravel.flight.flightlist.getflightlistrequest>
  )","path":"/flight-pc/api/flight/list"}
 </tw.com.eztravel.flight.flightlist.getflightlistrequest>
</tw.com.eztravel.flight.flightlist.getflightlistresponse>


In [111]:
import requests
# from bs4 import BeautifulSoup

url = 'https://flight.eztravel.com.tw/apiv2/flight/list?sig=6912de7ea4165a88'

response = requests.post(url)
# soup = BeautifulSoup(response.text, 'html.parser')
print(response.text)


{"timestamp":"2023-05-07T12:40:37.913+0000","status":400,"error":"Bad Request","message":"Required request body is missing: public tw.com.eztravel.flight.flightlist.CommonResponse<tw.com.eztravel.flight.flightlist.GetFlightListResponse> tw.com.eztravel.controller.FlightController.getFlightList(tw.com.eztravel.flight.flightlist.CommonRequest<tw.com.eztravel.flight.flightlist.GetFlightListRequest>)","path":"/flight-pc/api/flight/list"}


In [114]:
flights = []

flight_list = soup.find_all("div", class_="flight-list-detail")

for flight in flight_list:
    data = {
        'airline': flight.select_one('.airline').text.strip(),
        'departure_time': flight.select_one('.dep-time').text.strip(),
        'arrival_time': flight.select_one('.arr-time').text.strip(),
        'price': flight.select_one('.price').text.strip()
    }
    flights.append(data)
print(flight_list)

[]
